<div style="text-align: center; font-size: 16px;">
    <strong>Course:</strong> Machine Learning Operations |
    <strong>Lecturer:</strong> Prof. Dr. Klotz |
    <strong>Date:</strong> 17.05.2025 |
    <strong>Name:</strong> Sofie Pischl
</div>

# <center> Data Exploration </center>

# Inhaltsverzeichnis
1. [Einleitung](#Einleitung)
2. [Datenvorbereitung](#Datenvorbereitung)
3. [Fortgeschrittenes Topic Modelling mit BERTopic](#Fortgeschrittenes-Topic-Modelling-mit-BERTopic)
4. [Verbesserte Sentiment-Analyse](#Verbesserte-Sentiment-Analyse)
5. [Zeitliche Analyse der Topics](#Zeitliche-Analyse-der-Topics)
6. [UMAP-Visualisierung der Topics](#UMAP-Visualisierung-der-Topics)
7. [Vorbereitung auf Klassifikation](#Vorbereitung-auf-Klassifikation)


## Einleitung
Dieses Notebook untersucht kombinierte Social-Media-Daten aus Reddit, YouTube und TikTok. Ziel ist es, Muster in Themen und Stimmungen zu erkennen und die Daten für eine spätere Klassifikation vorzubereiten. Wir verwenden moderne Methoden wie **BERTopic**, **Transformer-basierte Sentimentanalyse** und **visuelle Embedding-Cluster**.

In diesem Notebook bauen wir auf der bisherigen Datenexploration auf und ergänzen folgende moderne Verfahren:

1. **Fortgeschrittenes Topic Modelling mit BERTopic**
2. **Verbesserte Sentiment-Analyse mit VADER und Transformer-Modell**
3. **Detaillierte Exploration von Text-Clustern und Zeitreihen**
4. **Vorbereitung für Klassifikation und Vorhersagemodelle**

Jede Methode wird durch beschreibenden Text begleitet und in das bestehende Datenformat integriert.

## Datenvorbereitung
Wir laden die vorverarbeiteten Daten, konvertieren Zeitstempel und bereiten die Texte für die Analyse vor.

In [98]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import nltk
from pathlib import Path

In [99]:
BASE_DIR = Path().resolve().parent
RAW_DIR = (BASE_DIR / "./data/processed").resolve()

# Datei laden
df = pd.read_csv(RAW_DIR / "social_media_data.csv")

display(df.head())
df.info()

,source,id,title,text,username,likes,comments,shares,plays,timestamp,published_at,url,datetime,date,title_language,title_clean,title_char_count,title_word_count,title_uppercase_count,title_exclamation_count,title_question_count,title_emoji_count,title_mention_count,title_hashtag_count,title_avg_word_length,title_sentiment,title_sentiment_score,text_language,text_clean,text_char_count,text_word_count,text_uppercase_count,text_exclamation_count,text_question_count,text_emoji_count,text_mention_count,text_hashtag_count,text_avg_word_length,text_sentiment,text_sentiment_score,engagement_rate,hour,weekday,year,month,day_period,is_weekend
0,tiktok,7493469801654881542,NaN,#vairalvideo_foryoupage #🇦🇫ازبک_تاجک_پشتون_تر...,afgcap.cut,0.000407,0.000566,0.000096,0.001126,2025-04-15 09:30:06+00:00,NaN,https://v16-webapp-prime.tiktok.com/video/tos/...,2025-04-15 09:30:06+00:00,2025-04-15,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,0.116029,NaN,NaN,NaN,NaN,NaN,False
1,tiktok,7489427780397010198,NaN,#imapoliceofficer #tensheet #foryou #viral #fy...,backwheelbandit69,0.110825,0.024776,0.064638,0.129241,2025-04-04 12:04:57+00:00,NaN,https://v16-webapp-prime.tiktok.com/video/tos/...,2025-04-04 12:04:57+00:00,2025-04-04,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,0.273845,NaN,NaN,NaN,NaN,NaN,False
2,tiktok,7492000423641959685,NaN,#CapCut #قوالب_كاب_كات_جاهزه_للتصميم__🌴♥ #كاب_...,noordeen_cap_cat_0_1,0.001505,0.000989,0.000335,0.004010,2025-04-11 10:28:10+00:00,NaN,https://v16-webapp-prime.tiktok.com/video/tos/...,2025-04-11 10:28:10+00:00,2025-04-11,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,0.119509,NaN,NaN,NaN,NaN,NaN,False
3,tiktok,7472584144510373125,NaN,i think it was a bad idea,maligoshik,0.463918,0.052621,0.276596,0.520049,2025-02-18 02:43:04+00:00,NaN,https://v16-webapp-prime.tiktok.com/video/tos/...,2025-02-18 02:43:04+00:00,2025-02-18,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,en,think bad idea,14.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,negative,-0.7,0.284582,NaN,NaN,NaN,NaN,NaN,False
4,tiktok,7461927005689302280,NaN,welcome to the thanos world!! #squidgame #squi...,team_thanos_player230,0.006737,0.028503,0.002027,0.038248,2025-01-20 09:27:52+00:00,NaN,https://webapp-sg.tiktok.com/bf63b8aa40b9ff0ca...,2025-01-20 09:27:52+00:00,2025-01-20,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,en,welcome thanos world,20.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,positive,0.8,0.057531,NaN,NaN,NaN,NaN,NaN,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   source                   1728 non-null   object 
 1   id                       1728 non-null   object 
 2   title                    646 non-null    object 
 3   text                     1727 non-null   object 
 4   username                 1398 non-null   object 
 5   likes                    1728 non-null   float64
 6   comments                 1728 non-null   float64
 7   shares                   1728 non-null   float64
 8   plays                    1728 non-null   float64
 9   timestamp                1560 non-null   object 
 10  published_at             645 non-null    object 
 11  url                      1304 non-null   object 
 12  datetime                 1726 non-null   object 
 13  date                     1726 non-null   object 
 14  title_language          

In [100]:
# Anzahl der Posts pro Plattform
post_counts = df["source"].value_counts().reset_index()
post_counts.columns = ["source", "num_posts"]

# Anzeige
print(post_counts)

    source  num_posts
0   tiktok       1082
1   reddit        330
2  youtube        316


# Preparations

In [101]:
# Kombinieren von Titel und Text (beide bereinigt)
texts = (df["title_clean"].fillna("") + " " + df["text_clean"].fillna("")).astype(str)

# LDA
LDA ist ein probabilistisches generatives Modell, das davon ausgeht, dass jedes Dokument eine Mischung aus mehreren Themen ist und jedes Thema durch eine Verteilung über Wörter beschrieben werden kann. Es eignet sich gut für längere, formellere Texte, bietet jedoch auch bei kurzen Texten erste Einblicke in thematische Strukturen.

Warum LDA als erster Ansatz?
Einfachheit & Transparenz: LDA ist leicht verständlich und bietet eine gute Grundlage, um Themenmodelle zu interpretieren.

Vergleichsbasis: Es dient als Referenzmodell, gegen das modernere Verfahren wie BERTopic verglichen werden können.

Schnelle Anwendung: Mit scikit-learn lässt sich LDA effizient auf vorbereiteten TF-IDF-Daten trainieren

In [102]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# CountVectorizer ohne Stopwords (da vorher entfernt)
vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2
)

# Textmatrix erzeugen
X = vectorizer.fit_transform(texts)  # texts = bereits bereinigte Texte

# LDA-Modell definieren
lda_model = LatentDirichletAllocation(
    n_components=9,         # Anzahl der Themen
    max_iter=3,
    learning_method='online',
    random_state=42
)

# LDA-Modell trainieren
lda_model.fit(X)


LatentDirichletAllocation(learning_method='online', max_iter=3, n_components=9,
                          random_state=42)

In [103]:
# Funktion zum Anzeigen der Themen mit Top-Wörtern
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"🟢 Thema {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print()

display_topics(lda_model, vectorizer.get_feature_names_out(), 10)


🟢 Thema 1:
would never baby asked year something last need back take

🟢 Thema 2:
kids dazn highlights sport live youtube klaas nhl besten the

🟢 Thema 3:
instagram youtube follow song twitter assistant production facebook music producer

🟢 Thema 4:
pf stl ast blk pts nbsp reb min ftm pm

🟢 Thema 5:
official league hair season highlights uefa champions make vs post

🟢 Thema 6:
like time get even people got us want know day

🟢 Thema 7:
video instagram tiktok mehr de kanal werbung geht facebook music

🟢 Thema 8:
said one told husband little really since feel stop aitah

🟢 Thema 9:
game tom team nba players points lead new search fg



## 1. Fortgeschrittenes Topic Modelling mit BERTopic

Das klassische LDA-Verfahren ist oft ungeeignet für kurze, informelle Texte. BERTopic nutzt Embeddings + Clustering + TF-IDF, um deutlich robustere und interpretierbare Themen zu erzeugen. Zudem unterstützt BERTopic die Analyse von Themen im Zeitverlauf.

In [104]:
# Installation (nur beim ersten Mal notwendig)
# !pip install bertopic[visualization] sentence-transformers

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

# Für deutschsprachige Texte kann ein mehrsprachiges Modell hilfreich sein
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # oder 'paraphrase-multilingual-MiniLM-L12-v2'

# Daten vorbereiten
texts = df['text_clean'].dropna().tolist()

# BERTopic initialisieren und trainieren
topic_model = BERTopic(embedding_model=embedding_model, language='multilingual')
topics, probs = topic_model.fit_transform(texts)

df_clean = df.loc[df['text_clean'].notna()].copy()
df_clean['topic'] = topics

topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,419,-1_instagram_tung_love_video,"[instagram, tung, love, video, music, like, yo...",[get exclusive nordvpn deal months extra risk ...
1,0,278,0_said_like_would_told,"[said, like, would, told, time, even, one, get...",[background f fiancé together years baby back ...
2,1,223,1_de_ik_en_het,"[de, ik, en, het, zijn, van, dat, die, ze, als]",[badania wykazują że nawet społeczeństwa myli ...
3,2,160,2_instagram_video_tiktok_werbung,"[instagram, video, tiktok, werbung, youtube, k...",[spaß video freunde original video sei live da...
4,3,90,3_wait_omg_ok_laugh,"[wait, omg, ok, laugh, end, description, sweet...","[wait, wait, wait end]"
5,4,45,4_brainrot_tragic_italian_cat,"[brainrot, tragic, italian, cat, song, speedru...",[tralalero tralala x wakawaka brainrot movie t...
6,5,40,5_league_dazn_highlights_uefa,"[league, dazn, highlights, uefa, champions, sp...",[königlichen gast beim tabellenzwöflten getafe...
7,6,34,6_pf_pts_ast_stl,"[pf, pts, ast, stl, blk, nbsp, min, reb, stats...",[gsw min box scores nba yahoo nbsp game summar...
8,7,33,7_اللي_انا_official_في,"[اللي, انا, official, في, والمزيكا, على, زفة, ...",[academy σας περιμένουμε και στα υπόλοιπα κατα...
9,8,31,8_production_assistant_director_producer,"[production, assistant, director, producer, mu...",[spieglein spieglein jetzt hören spieglein spi...


## 2. Verbesserte Sentiment-Analyse

Die bisher genutzte Methode (`TextBlob`) ist eher für formellere englische Texte geeignet. Hier ergänzen wir zwei modernere Verfahren:

- **VADER**: speziell für Social Media-Texte (auch kurze Aussagen) konzipiert.
- **Transformer-Modell**: z. B. RoBERTa-basiert, feingetunt für Sentiment-Klassifikation auf Twitter-Daten.

In [109]:
from transformers import pipeline

# Sentiment-Pipeline laden (Roberta Twitter-Modell)
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True
)

# Einzelne Zeilen durchgehen, um Fehler im Batch zu vermeiden
def safe_sentiment(text):
    try:
        return sentiment_pipeline(text[:512])[0]['label']
    except Exception as e:
        print(f"Fehler bei Text: {text[:100]}... -> {e}")
        return "error"

# Auf eine Teilmenge testen (z. B. 500 Zeilen)
# df_clean = df_clean.sample(n=500, random_state=42).copy()

df_clean['roberta_sentiment'] = df_clean['text_clean'].astype(str).apply(safe_sentiment)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Fehler bei Text: 𝗙𝗢𝗟𝗘 𝗣𝗨𝗕𝗟𝗜𝗦𝗛𝗜𝗡𝗚 ____________________________ 𝗫𝗵𝗲𝗻𝘀𝗶𝗹𝗮 𝘅 𝗟𝗲𝗱𝗿𝗶 𝗩𝘂𝗹𝗮 𝗠𝗮 𝗞𝘁𝗵𝗲 𝗢𝗳𝗳𝗶𝗰𝗶𝗮𝗹 𝗩𝗶𝗱𝗲𝗼 __________... -> The expanded size of the tensor (582) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 582].  Tensor sizes: [1, 514]
Fehler bei Text: اشترك في قناة حسين الجسمي الرسمية حسين الجسمي فستانك الأبيض استمعوا على جميع المنصات كلمات أمير طعيم... -> The expanded size of the tensor (516) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 516].  Tensor sizes: [1, 514]


## 3. Zeitliche Analyse der Topics

Ein großer Vorteil von BERTopic ist die Möglichkeit, Themen im Zeitverlauf zu analysieren. Dazu müssen wir das Erstellungsdatum korrekt als `datetime` interpretieren und in geeignete Zeitstempel transformieren.

In [110]:
# Sicherstellen, dass timestamp eine gültige datetime-Spalte ist
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'], errors='coerce')
df_clean = df_clean.dropna(subset=['timestamp'])

# Alle Listen synchron extrahieren
texts_synced = df_clean['text_clean'].tolist()
topics_synced = df_clean['topic'].tolist()
timestamps_synced = df_clean['timestamp'].tolist()  # echte datetime-Werte

# Kontrollieren
print(len(texts_synced), len(topics_synced), len(timestamps_synced))

# Topics-over-time berechnen (echte Timestamps → funktioniert!)
topics_over_time = topic_model.topics_over_time(
    docs=texts_synced,
    topics=topics_synced,
    timestamps=timestamps_synced,
    nr_bins=20  # oder eine andere sinnvolle Anzahl Zeit-Buckets
)

# Visualisierung
topic_model.visualize_topics_over_time(topics_over_time)


887 887 887


## 4. Visuelle Exploration mit UMAP

Zur besseren Interpretation der durch BERTopic erzeugten Embeddings und Topics projizieren wir die hochdimensionalen Embeddings in 2D mit **UMAP**. So lassen sich thematische Cluster visuell erkennen.

In [ ]:

# Visualisierung nach Topic
topic_model.visualize_documents(texts, topics=topics)

#  Save

In [112]:
df_clean.to_csv(RAW_DIR / "social_media_data_with_topics.csv")


In [113]:
df_clean.head()

,source,id,title,text,username,likes,comments,shares,plays,timestamp,published_at,url,datetime,date,title_language,title_clean,title_char_count,title_word_count,title_uppercase_count,title_exclamation_count,title_question_count,title_emoji_count,title_mention_count,title_hashtag_count,title_avg_word_length,title_sentiment,title_sentiment_score,text_language,text_clean,text_char_count,text_word_count,text_uppercase_count,text_exclamation_count,text_question_count,text_emoji_count,text_mention_count,text_hashtag_count,text_avg_word_length,text_sentiment,text_sentiment_score,engagement_rate,hour,weekday,year,month,day_period,is_weekend,topic,vader_score,roberta_sentiment
3,tiktok,7472584144510373125,NaN,i think it was a bad idea,maligoshik,0.463918,0.052621,0.276596,0.520049,2025-02-18 02:43:04+00:00,NaN,https://v16-webapp-prime.tiktok.com/video/tos/...,2025-02-18 02:43:04+00:00,2025-02-18,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,en,think bad idea,14.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,negative,-0.7,0.284582,NaN,NaN,NaN,NaN,NaN,False,3,-0.5423,LABEL_0
4,tiktok,7461927005689302280,NaN,welcome to the thanos world!! #squidgame #squi...,team_thanos_player230,0.006737,0.028503,0.002027,0.038248,2025-01-20 09:27:52+00:00,NaN,https://webapp-sg.tiktok.com/bf63b8aa40b9ff0ca...,2025-01-20 09:27:52+00:00,2025-01-20,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,en,welcome thanos world,20.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,positive,0.8,0.057531,NaN,NaN,NaN,NaN,NaN,False,-1,0.4588,LABEL_2
5,tiktok,7461757350492278048,NaN,This is a great goal 😳😭 @Fabio Andrés #golazo ...,risingballers,0.022387,0.012754,0.049234,0.033621,2025-01-19 22:29:34+00:00,NaN,https://v16-webapp-prime.tiktok.com/video/tos/...,2025-01-19 22:29:34+00:00,2025-01-19,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,en,great goal andrés,17.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,positive,0.8,0.213120,NaN,NaN,NaN,NaN,NaN,False,13,0.6249,LABEL_2
6,tiktok,7462429961345879318,NaN,glaube er hatte nicht so bock auf mich..💀 #for...,allroundii,0.017598,0.015076,0.035149,0.031462,2025-01-21 17:59:37+00:00,NaN,NaN,2025-01-21 17:59:37+00:00,2025-01-21,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,de,glaube bock,11.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,neutral,0.0,0.179356,NaN,NaN,NaN,NaN,NaN,False,1,0.0000,LABEL_1
7,tiktok,7483193151004445974,NaN,#CapCut Speedquiz 🤯🧠✅️ #fyp #viral_video #quiz...,brainyy.quiz,0.000719,0.016347,0.004191,0.003393,2025-03-18 16:51:23+00:00,NaN,NaN,2025-03-18 16:51:23+00:00,2025-03-18,unknown,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,neutral,0.0,nl,speedquiz,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,neutral,0.0,0.084348,NaN,NaN,NaN,NaN,NaN,False,-1,0.0000,LABEL_1
